# NBA Data Gathering

In [35]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import sqlalchemy as db
import os
import re

## Data Gathering

### Nba Stats per Season (Web Scrapping)

In [93]:
url = "https://www.basketball-reference.com/leagues/NBA_stats_per_game.html"
response = requests.get(url)
soup = BeautifulSoup(response.content)
table = soup.find_all("table", attrs={"id":"stats"})[0]

In [94]:
headers = [header.text.strip() for header in table.find_all('th',attrs={"scope":"col"})][1:]
headers

['Season',
 'Lg',
 'Age',
 'Ht',
 'Wt',
 'G',
 'MP',
 'FG',
 'FGA',
 '3P',
 '3PA',
 'FT',
 'FTA',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS',
 'FG%',
 '3P%',
 'FT%',
 'Pace',
 'eFG%',
 'TOV%',
 'ORB%',
 'FT/FGA',
 'ORtg']

In [95]:
data = [data.text.strip() for data in table.find_all('td')]
data

['2019-20',
 'NBA',
 '26.0',
 '6-6',
 '216',
 '971',
 '241.7',
 '40.8',
 '88.8',
 '12.1',
 '33.9',
 '17.7',
 '22.9',
 '10.1',
 '34.7',
 '44.9',
 '24.3',
 '7.7',
 '4.9',
 '14.5',
 '20.6',
 '111.4',
 '.460',
 '.357',
 '.771',
 '100.2',
 '.528',
 '12.8',
 '22.6',
 '.199',
 '110.4',
 '2018-19',
 'NBA',
 '26.3',
 '6-6',
 '217',
 '1230',
 '241.6',
 '41.1',
 '89.2',
 '11.4',
 '32.0',
 '17.7',
 '23.1',
 '10.3',
 '34.8',
 '45.2',
 '24.6',
 '7.6',
 '5.0',
 '14.1',
 '20.9',
 '111.2',
 '.461',
 '.355',
 '.766',
 '100.0',
 '.524',
 '12.4',
 '22.9',
 '.198',
 '110.4',
 '2017-18',
 'NBA',
 '26.4',
 '6-7',
 '219',
 '1230',
 '241.4',
 '39.6',
 '86.1',
 '10.5',
 '29.0',
 '16.6',
 '21.7',
 '9.7',
 '33.8',
 '43.5',
 '23.2',
 '7.7',
 '4.8',
 '14.3',
 '19.9',
 '106.3',
 '.460',
 '.362',
 '.767',
 '97.3',
 '.521',
 '13.0',
 '22.3',
 '.193',
 '108.6',
 '2016-17',
 'NBA',
 '26.6',
 '6-7',
 '220',
 '1230',
 '241.6',
 '39.0',
 '85.4',
 '9.7',
 '27.0',
 '17.8',
 '23.1',
 '10.1',
 '33.4',
 '43.5',
 '22.6',
 '7.7',

In [96]:
ncols = len(headers)
nrows = int(len(data)/ncols)

In [97]:
seasons = pd.DataFrame(np.array(data).reshape((nrows, ncols)), columns=headers)

In [100]:
#full table from basketball reference, stats are per game
pd.set_option('max_columns', 32)
seasons.head(30)

Season   Lg   Age   Ht   Wt     G     MP    FG   FGA    3P   3PA    FT  \
0   2019-20  NBA  26.0  6-6  216   971  241.7  40.8  88.8  12.1  33.9  17.7   
1   2018-19  NBA  26.3  6-6  217  1230  241.6  41.1  89.2  11.4  32.0  17.7   
2   2017-18  NBA  26.4  6-7  219  1230  241.4  39.6  86.1  10.5  29.0  16.6   
3   2016-17  NBA  26.6  6-7  220  1230  241.6  39.0  85.4   9.7  27.0  17.8   
4   2015-16  NBA  26.7  6-7  221  1230  241.8  38.2  84.6   8.5  24.1  17.7   
5   2014-15  NBA  26.7  6-7  222  1230  242.0  37.5  83.6   7.8  22.4  17.1   
6   2013-14  NBA  26.5  6-7  223  1230  242.0  37.7  83.0   7.7  21.5  17.8   
7   2012-13  NBA  26.7  6-7  223  1229  241.9  37.1  82.0   7.2  20.0  16.7   
8   2011-12  NBA  26.6  6-7  223   990  241.9  36.5  81.4   6.4  18.4  16.9   
9   2010-11  NBA  26.6  6-7  223  1230  241.9  37.2  81.2   6.5  18.0  18.6   
10  2009-10  NBA  26.6  6-7  222  1230  241.7  37.7  81.7   6.4  18.1  18.6   
11  2008-09  NBA  26.6  6-7  221  1230  241.7  37.1  80.9   6.6  18.1  19.1   
12  2007-08  NBA  26.8  6-7  220  1230  241.5  37.3  81.5   6.6  18.1  18.8   
13  2006-07  NBA  26.6  6-7  219  1230  242.2  36.5  79.7   6.1  16.9  19.6   
14  2005-06  NBA  26.5  6-7  220  1230  242.1  35.8  79.0   5.7  16.0  19.6   
15  2004-05  NBA  26.9  6-7  220  1230  241.9  35.9  80.3   5.6  15.8  19.7   
16  2003-04  NBA  27.0  6-7  219  1189  241.7  35.0  79.8   5.2  14.9  18.2   
17  2002-03  NBA  27.2  6-7  219  1189  242.0  35.7  80.8   5.1  14.7  18.5   
18  2001-02  NBA  27.4  6-7  218  1189  241.7  36.2  81.3   5.2  14.7  17.9   
19  2000-01  NBA  27.7  6-7  216  1189  242.0  35.7  80.6   4.8  13.7  18.6   
20  1999-00  NBA  27.8  6-7  216  1189  241.5  36.8  82.1   4.8  13.7  19.0   
21  1998-99  NBA  27.9  6-7  215   725  241.8  34.2  78.2   4.5  13.2  18.8   
22  1997-98  NBA  27.7  6-7  214  1189  241.9  35.9  79.7   4.4  12.7  19.4   
23  1996-97  NBA  27.7  6-7  213  1189  241.9  36.1  79.3   6.0  16.8  18.7   
24  1995-96  NBA  27.5  6-7  213  1189  241.6  37.0  80.2   5.9  16.0  19.5   
25  1994-95  NBA  27.2  6-7  212  1107  241.9  38.0  81.5   5.5  15.3  19.9   
26  1993-94  NBA  27.2  6-7  211  1107  241.1  39.3  84.4   3.3   9.9  19.6   
27  1992-93  NBA  27.1  6-7  210  1107  241.7  40.7  86.0   3.0   9.0  20.9   
28  1991-92  NBA  27.2  6-7  208  1107  241.8  41.3  87.3   2.5   7.6  20.2   
29  1990-91  NBA  27.2  6-7  209  1107  241.8  41.4  87.2   2.3   7.1  21.3   

     FTA   ORB   DRB   TRB   AST  STL  BLK   TOV    PF    PTS   FG%   3P%  \
0   22.9  10.1  34.7  44.9  24.3  7.7  4.9  14.5  20.6  111.4  .460  .357   
1   23.1  10.3  34.8  45.2  24.6  7.6  5.0  14.1  20.9  111.2  .461  .355   
2   21.7   9.7  33.8  43.5  23.2  7.7  4.8  14.3  19.9  106.3  .460  .362   
3   23.1  10.1  33.4  43.5  22.6  7.7  4.7  14.0  19.9  105.6  .457  .358   
4   23.4  10.4  33.3  43.8  22.3  7.8  5.0  14.4  20.3  102.7  .452  .354   
5   22.8  10.9  32.4  43.3  22.0  7.7  4.8  14.4  20.2  100.0  .449  .350   
6   23.6  10.9  31.8  42.7  22.0  7.7  4.7  14.6  20.7  101.0  .454  .360   
7   22.2  11.2  31.0  42.1  22.1  7.8  5.1  14.6  19.8   98.1  .453  .359   
8   22.5  11.4  30.8  42.2  21.0  7.7  5.1  14.6  19.6   96.3  .448  .349   
9   24.4  10.9  30.5  41.4  21.5  7.3  4.9  14.3  20.7   99.6  .459  .358   
10  24.5  11.0  30.8  41.7  21.2  7.2  4.9  14.2  20.9  100.4  .461  .355   
11  24.7  11.0  30.3  41.3  21.0  7.3  4.8  14.0  21.0  100.0  .459  .367   
12  24.9  11.2  30.8  42.0  21.8  7.3  4.7  14.1  21.0   99.9  .457  .362   
13  26.1  11.1  29.9  41.1  21.3  7.2  4.6  15.1  22.2   98.7  .458  .358   
14  26.3  11.2  29.8  41.0  20.6  7.2  4.7  14.4  22.8   97.0  .454  .358   
15  26.1  12.0  29.8  41.9  21.3  7.5  4.9  14.5  22.6   97.2  .447  .356   
16  24.2  12.1  30.1  42.2  21.3  7.9  5.1  15.0  21.4   93.4  .439  .347   
17  24.4  12.0  30.3  42.3  21.5  7.9  5.0  14.9  21.8   95.1  .442  .349   
18  23.8  12.2  30.2  42.4  21.9  7.8  5.2  14.5  21.2   95.5  .445  .354   
19  24.9  12.0 

### NBA Stats per players (API)

#### All players database

In [70]:
response=requests.get("https://www.balldontlie.io/api/v1/players?per_page=100&page=1")
response

<Response [200]>

In [71]:
pagesrange=range(1,34)

In [72]:
# full table of all players from balldontlie API
players = pd.DataFrame()
for i in pagesrange:
    playerpage=pd.DataFrame(requests.get(f"https://www.balldontlie.io/api/v1/players?per_page=100&page={i}").json()["data"])
    players=pd.concat([players,playerpage])
players.head()

id first_name  height_feet  height_inches last_name position  \
0  14        Ike          NaN            NaN  Anigbogu        C   
1  25        Ron          NaN            NaN     Baker        G   
2  47     Jabari          NaN            NaN      Bird        G   
3  67    MarShon          NaN            NaN    Brooks        G   
4  71    Lorenzo          NaN            NaN     Brown        G   

                                                team  weight_pounds  
0  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...            NaN  
1  {'id': 20, 'abbreviation': 'NYK', 'city': 'New...            NaN  
2  {'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...            NaN  
3  {'id': 15, 'abbreviation': 'MEM', 'city': 'Mem...            NaN  
4  {'id': 28, 'abbreviation': 'TOR', 'city': 'Tor...            NaN

In [73]:
players.loc[players["last_name"]=="Carter",:]

id first_name  height_feet  height_inches last_name position  \
53  1224    Anthony          NaN            NaN    Carter            
51  1523    Maurice          NaN            NaN    Carter            
59  2469        Ron          NaN            NaN    Carter            
20  2530     Reggie          NaN            NaN    Carter            
21  2531      Butch          NaN            NaN    Carter            
46  2656     Howard          NaN            NaN    Carter            
36    87      Jevon          6.0            2.0    Carter        G   
37    88      Vince          6.0            6.0    Carter      F-G   

                                                 team  weight_pounds  
53  {'id': 16, 'abbreviation': 'MIA', 'city': 'Mia...            NaN  
51  {'id': 14, 'abbreviation': 'LAL', 'city': 'Los...            NaN  
59  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...            NaN  
20  {'id': 20, 'abbreviation': 'NYK', 'city': 'New...            NaN  
21  {'id': 14, 'abbreviation': 'LAL', 'city': 'Los...            NaN  
46  {'id': 8, 'abbreviation': 'DEN', 'city': 'Denv...            NaN  
36  {'id': 24, 'abbreviation': 'PHX', 'city': 'Pho...          196.0  
37  {'id': 1, 'abbreviation': 'ATL', 'city': 'Atla...          220.0

In [74]:
players.shape

(3268, 8)

#### Seasons Average database

In [13]:
response = requests.get("https://www.balldontlie.io/api/v1/season_averages?season=1979&player_ids[]=577")
response.json()

{'data': []}

In [14]:
seasons_range=list(range(2010,2019))
seasons_range

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018]

In [15]:
playersidrange = list(players["id"].unique())
len(playersidrange)

3268

In [16]:
playerseason=pd.DataFrame(requests.get(f"https://www.balldontlie.io/api/v1/season_averages?player_ids[]=115&season=2016").json()["data"])
playerseason

games_played  player_id  season    min   fgm    fga  fg3m  fg3a   ftm  \
0            79        115    2016  33:23  8.54  18.27   4.1  9.99  4.11   

    fta  oreb  dreb   reb   ast  stl   blk  turnover    pf   pts  fg_pct  \
0  4.58  0.77   3.7  4.47  6.63  1.8  0.22      3.03  2.32  25.3   0.468   

   fg3_pct  ft_pct  
0    0.411   0.898

In [17]:
players_season_average_2010s = pd.DataFrame()
for s in seasons_range:
    for f in tqdm(playersidrange):
        try:
            playerseason=pd.DataFrame(requests.get(f"https://www.balldontlie.io/api/v1/season_averages?season={s}&player_ids[]={f}").json()["data"])
            players_season_average_2010s=pd.concat([players_season_average_2010s,playerseason])
        except:
            pass
players_season_average_2010s.head()

games_played  player_id  season    min   fgm    fga  fg3m  fg3a   ftm  \
0            71         90    2010  23:58  3.17   7.69  1.30  3.48  0.96   
0            80        179    2010  25:24  4.23   7.54  0.01  0.05  1.75   
0            65       1593    2010  35:20  5.65  11.42  0.00  0.06  1.48   
0            72        241    2010  25:43  3.90   6.88  0.00  0.01  1.75   
0            50        263    2010   8:40  1.36   3.00  0.00  0.04  0.48   

    fta  oreb  dreb    reb   ast   stl   blk  turnover    pf    pts  fg_pct  \
0  1.42  1.04  3.28   4.32  1.04  0.76  0.18      0.96  1.62   8.59   0.412   
0  2.41  1.85  6.04   7.89  0.88  0.45  1.14      1.01  2.35  10.21   0.561   
0  3.34  3.06  8.02  11.08  2.02  0.72  2.58      1.92  3.34  12.77   0.495   
0  2.22  2.63  3.78   6.40  1.13  0.74  1.22      0.96  3.65   9.56   0.568   
0  0.86  1.14  1.46   2.60  0.14  0.20  0.50      0.62  1.18   3.20   0.453   

   fg3_pct  ft_pct  
0    0.372   0.673  
0    0.250   0.725  
0    0.000   0.442  
0    0.000   0.788  
0    0.000   0.558

## Data Cleaning and Manipulation

### NBA Stats per Season

In [101]:
seasons

Season   Lg   Age   Ht   Wt     G     MP    FG   FGA    3P   3PA    FT  \
0   2019-20  NBA  26.0  6-6  216   971  241.7  40.8  88.8  12.1  33.9  17.7   
1   2018-19  NBA  26.3  6-6  217  1230  241.6  41.1  89.2  11.4  32.0  17.7   
2   2017-18  NBA  26.4  6-7  219  1230  241.4  39.6  86.1  10.5  29.0  16.6   
3   2016-17  NBA  26.6  6-7  220  1230  241.6  39.0  85.4   9.7  27.0  17.8   
4   2015-16  NBA  26.7  6-7  221  1230  241.8  38.2  84.6   8.5  24.1  17.7   
..      ...  ...   ...  ...  ...   ...    ...   ...   ...   ...   ...   ...   
69  1950-51  NBA                   354         29.8  83.6              24.5   
70  1949-50  NBA                   561         28.2  83.1              23.6   
71  1948-49  BAA                   360         29.0  88.7              22.0   
72  1947-48  BAA                   192         27.2  96.0              18.2   
73  1946-47  BAA                   331         25.9  92.9              15.9   

     FTA   ORB   DRB   TRB   AST  STL  BLK   TOV    PF    PTS   FG%   3P%  \
0   22.9  10.1  34.7  44.9  24.3  7.7  4.9  14.5  20.6  111.4  .460  .357   
1   23.1  10.3  34.8  45.2  24.6  7.6  5.0  14.1  20.9  111.2  .461  .355   
2   21.7   9.7  33.8  43.5  23.2  7.7  4.8  14.3  19.9  106.3  .460  .362   
3   23.1  10.1  33.4  43.5  22.6  7.7  4.7  14.0  19.9  105.6  .457  .358   
4   23.4  10.4  33.3  43.8  22.3  7.8  5.0  14.4  20.3  102.7  .452  .354   
..   ...   ...   ...   ...   ...  ...  ...   ...   ...    ...   ...   ...   
69  33.4              49.5  21.0                  27.0   84.1  .357         
70  33.0                    19.6                  27.0   80.0  .340         
71  31.3                    18.6                  25.6   80.0  .327         
72  27.0                     7.0                  22.2   72.7  .284         
73  24.8                     6.9                  20.8   67.8  .279         

     FT%   Pace  eFG%  TOV%  ORB% FT/FGA   ORtg  
0   .771  100.2  .528  12.8  22.6   .199  110.4  
1   .766  100.0  .524  12.4  22.9   .198  110.4  
2   .767   97.3  .521  13.0  22.3   .193  108.6  
3   .772   96.4  .514  12.7  23.3   .209  108.8  
4   .757   95.8  .502  13.2  23.8   .209  106.4  
..   ...    ...   ...   ...   ...    ...    ...  
69  .733         .357               .293         
70  .714         .340               .284         
71  .703         .327               .248         
72  .675         .284               .190         
73  .641         .279               .171         

[74 rows x 31 columns]

In [102]:
seasons.dtypes

Season    object
Lg        object
Age       object
Ht        object
Wt        object
G         object
MP        object
FG        object
FGA       object
3P        object
3PA       object
FT        object
FTA       object
ORB       object
DRB       object
TRB       object
AST       object
STL       object
BLK       object
TOV       object
PF        object
PTS       object
FG%       object
3P%       object
FT%       object
Pace      object
eFG%      object
TOV%      object
ORB%      object
FT/FGA    object
ORtg      object
dtype: object

In [103]:
#Convert Ht to meters
seasons["HtInches"]=seasons["Ht"].apply(lambda x: "" if re.findall("-\d{1,2}",str(x))==[] else re.findall("-\d{1,2}",str(x))[0])
seasons["HtInches"]=seasons["HtInches"].apply(lambda x: "" if re.findall("\d{1,2}",str(x))==[] else re.findall("\d{1,2}",str(x))[0])
seasons["HtFeet"]=seasons["Ht"].apply(lambda x: str(x).split("-")[0].replace("nan",""))
seasons["HtFeet"]=seasons["HtFeet"].replace(r'^\s*$', np.nan, regex=True)
seasons["HtInches"]=seasons["HtFeet"].replace(r'^\s*$', np.nan, regex=True)
seasons["HtFeet"]=seasons["HtFeet"].astype(float)
seasons["HtInches"]=seasons["HtInches"].astype(float)

In [104]:
seasons["Ht"]=seasons["HtFeet"]*0.3048+seasons["HtInches"]*0.025

In [115]:
seasons=seasons.drop(columns=['HtFeet', 'HtInches'])

In [106]:
seasons.head(20)

Season   Lg   Age      Ht   Wt     G     MP    FG   FGA    3P   3PA  \
0   2019-20  NBA  26.0  1.9788  216   971  241.7  40.8  88.8  12.1  33.9   
1   2018-19  NBA  26.3  1.9788  217  1230  241.6  41.1  89.2  11.4  32.0   
2   2017-18  NBA  26.4  1.9788  219  1230  241.4  39.6  86.1  10.5  29.0   
3   2016-17  NBA  26.6  1.9788  220  1230  241.6  39.0  85.4   9.7  27.0   
4   2015-16  NBA  26.7  1.9788  221  1230  241.8  38.2  84.6   8.5  24.1   
5   2014-15  NBA  26.7  1.9788  222  1230  242.0  37.5  83.6   7.8  22.4   
6   2013-14  NBA  26.5  1.9788  223  1230  242.0  37.7  83.0   7.7  21.5   
7   2012-13  NBA  26.7  1.9788  223  1229  241.9  37.1  82.0   7.2  20.0   
8   2011-12  NBA  26.6  1.9788  223   990  241.9  36.5  81.4   6.4  18.4   
9   2010-11  NBA  26.6  1.9788  223  1230  241.9  37.2  81.2   6.5  18.0   
10  2009-10  NBA  26.6  1.9788  222  1230  241.7  37.7  81.7   6.4  18.1   
11  2008-09  NBA  26.6  1.9788  221  1230  241.7  37.1  80.9   6.6  18.1   
12  2007-08  NBA  26.8  1.9788  220  1230  241.5  37.3  81.5   6.6  18.1   
13  2006-07  NBA  26.6  1.9788  219  1230  242.2  36.5  79.7   6.1  16.9   
14  2005-06  NBA  26.5  1.9788  220  1230  242.1  35.8  79.0   5.7  16.0   
15  2004-05  NBA  26.9  1.9788  220  1230  241.9  35.9  80.3   5.6  15.8   
16  2003-04  NBA  27.0  1.9788  219  1189  241.7  35.0  79.8   5.2  14.9   
17  2002-03  NBA  27.2  1.9788  219  1189  242.0  35.7  80.8   5.1  14.7   
18  2001-02  NBA  27.4  1.9788  218  1189  241.7  36.2  81.3   5.2  14.7   
19  2000-01  NBA  27.7  1.9788  216  1189  242.0  35.7  80.6   4.8  13.7   

      FT   FTA   ORB   DRB   TRB  ...  STL  BLK   TOV    PF    PTS   FG%  \
0   17.7  22.9  10.1  34.7  44.9  ...  7.7  4.9  14.5  20.6  111.4  .460   
1   17.7  23.1  10.3  34.8  45.2  ...  7.6  5.0  14.1  20.9  111.2  .461   
2   16.6  21.7   9.7  33.8  43.5  ...  7.7  4.8  14.3  19.9  106.3  .460   
3   17.8  23.1  10.1  33.4  43.5  ...  7.7  4.7  14.0  19.9  105.6  .457   
4   17.7  23.4  10.4  33.3  43.8  ...  7.8  5.0  14.4  20.3  102.7  .452   
5   17.1  22.8  10.9  32.4  43.3  ...  7.7  4.8  14.4  20.2  100.0  .449   
6   17.8  23.6  10.9  31.8  42.7  ...  7.7  4.7  14.6  20.7  101.0  .454   
7   16.7  22.2  11.2  31.0  42.1  ...  7.8  5.1  14.6  19.8   98.1  .453   
8   16.9  22.5  11.4  30.8  42.2  ...  7.7  5.1  14.6  19.6   96.3  .448   
9   18.6  24.4  10.9  30.5  41.4  ...  7.3  4.9  14.3  20.7   99.6  .459   
10  18.6  24.5  11.0  30.8  41.7  ...  7.2  4.9  14.2  20.9  100.4  .461   
11  19.1  24.7  11.0  30.3  41.3  ...  7.3  4.8  14.0  21.0  100.0  .459   
12  18.8  24.9  11.2  30.8  42.0  ...  7.3  4.7  14.1  21.0   99.9  .457   
13  19.6  26.1  11.1  29.9  41.1  ...  7.2  4.6  15.1  22.2   98.7  .458   
14  19.6  26.3  11.2  29.8  41.0  ...  7.2  4.7  14.4  22.8   97.0  .454   
15  19.7  26.1  12.0  29.8  41.9  ...  7.5  4.9  14.5  22.6   97.2  .447   
16  18.2  24.2  12.1  30.1  42.2  ...  7.9  5.1  15.0  21.4   93.4  .439   
17  18.5  24.4  12.0  30.3  42.3  ...  7.9  5.0  14.9  21.8   95.1  .442   
18  17.9  23.8  12.2  30.2  42.4  ...  7.8  5.2  14.5  21.2   95.5  .445   
19  18.6  24.9  12.0  30.5  42.5  ...  7.8  5.3  15.0  22.3   94.8  .443   

     3P%   FT%   Pace  eFG%  TOV%  ORB% FT/FGA   ORtg HtInches  HtFeet  
0   .357  .771  100.2  .528  12.8  22.6   .199  110.4      6.0     6.0  
1   .355  .766  100.0  .524  12.4  22.9   .198  110.4      6.0     6.0  
2   .362  .767   97.3  .521  13.0  22.3   .193  108.6      6.0     6.0  
3   .358  .772   96.4  .514  12.7  23.3   .209  108.8      6.0     6.0  
4   .354  .757   95.8  .502  13.2  23.8   .209  106.4      6.0     6.0  
5   .350  .750   93.9  .496  13.3  25.1   .205  105.6      6.0     6.0  
6   .360  .756   93.9  .501  13.6  25.5   .215  106.6      6.0     6.0  
7   .359  .753   92.0  .496  13.7  26.5   .204  105.8      6.0     6.0  
8   .349  .752   91.3  .487  13.8  27.0   .208  104.6      6.0     6.0  
9   .358  .763   92.1  .498  13.4  26.4   .229  107.3      6.0     6.0  
10  .355

In [107]:
#replacing empty values for nan
seasons=seasons.replace(r'^\s*$', np.nan, regex=True)

In [108]:
seasons["Season"]=seasons.loc[:,"Season"].apply(lambda x: x[0:4])

In [110]:
seasons["Season"]=seasons["Season"].astype(int)
seasons["Age"]=seasons["Age"].astype(float)
seasons["Ht"]=seasons["Ht"].astype(float)
seasons["Wt"]=seasons["Wt"].astype(float)
seasons["G"]=seasons["G"].astype(int)
seasons["MP"]=seasons["MP"].astype(float)
seasons["FG"]=seasons["FG"].astype(float)
seasons["FGA"]=seasons["FGA"].astype(float)
seasons["3P"]=seasons["3P"].astype(float)
seasons["3PA"]=seasons["3PA"].astype(float)
seasons["FT"]=seasons["FT"].astype(float)
seasons["FTA"]=seasons["FTA"].astype(float)
seasons["ORB"]=seasons["ORB"].astype(float)
seasons["DRB"]=seasons["DRB"].astype(float)
seasons["TRB"]=seasons["TRB"].astype(float)
seasons["AST"]=seasons["AST"].astype(float)
seasons["STL"]=seasons["STL"].astype(float)
seasons["BLK"]=seasons["BLK"].astype(float)
seasons["TOV"]=seasons["TOV"].astype(float)
seasons["PF"]=seasons["PF"].astype(float)
seasons["PTS"]=seasons["PTS"].astype(float)
seasons["FG%"]=seasons["FG%"].astype(float)
seasons["3P%"]=seasons["3P%"].astype(float)
seasons["FT%"]=seasons["FT%"].astype(float)
seasons["Pace"]=seasons["Pace"].astype(float)
seasons["eFG%"]=seasons["eFG%"].astype(float)
seasons["TOV%"]=seasons["TOV%"].astype(float)
seasons["ORB%"]=seasons["ORB%"].astype(float)
seasons["FT/FGA"]=seasons["FT/FGA"].astype(float)
seasons["ORtg"]=seasons["ORtg"].astype(float)

In [168]:
seasons=seasons.rename(columns={"FG%":"FG_pct","3P%":"3P_pct","FT%":"FT_pct","eFG%":"eFG_pct","TOV%":"TOV_pct","eFG%":"eFG_pct","TOV%":"TOV_pct","ORB%":"ORB_pct",})

In [116]:
seasons.dtypes

Season      int32
Lg         object
Age       float64
Ht        float64
Wt        float64
G           int32
MP        float64
FG        float64
FGA       float64
3P        float64
3PA       float64
FT        float64
FTA       float64
ORB       float64
DRB       float64
TRB       float64
AST       float64
STL       float64
BLK       float64
TOV       float64
PF        float64
PTS       float64
FG%       float64
3P%       float64
FT%       float64
Pace      float64
eFG%      float64
TOV%      float64
ORB%      float64
FT/FGA    float64
ORtg      float64
dtype: object

### NBA Stats per players

In [112]:
players

id first_name  height_feet  height_inches         last_name position  \
0       14        Ike          NaN            NaN          Anigbogu        C   
1       25        Ron          NaN            NaN             Baker        G   
2       47     Jabari          NaN            NaN              Bird        G   
3       67    MarShon          NaN            NaN            Brooks        G   
4       71    Lorenzo          NaN            NaN             Brown        G   
..     ...        ...          ...            ...               ...      ...   
63  666641     Jaylen          NaN            NaN             Hoard        F   
64     213      Aaron          6.0            1.0           Holiday        G   
65     214       Jrue          6.0            4.0           Holiday        G   
66     215     Justin          6.0            6.0           Holiday      F-G   
67     216     Rondae          6.0            7.0  Hollis-Jefferson        F   

                                                 team  weight_pounds  
0   {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...            NaN  
1   {'id': 20, 'abbreviation': 'NYK', 'city': 'New...            NaN  
2   {'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...            NaN  
3   {'id': 15, 'abbreviation': 'MEM', 'city': 'Mem...            NaN  
4   {'id': 28, 'abbreviation': 'TOR', 'city': 'Tor...            NaN  
..                                                ...            ...  
63  {'id': 25, 'abbreviation': 'POR', 'city': 'Por...            NaN  
64  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...          185.0  
65  {'id': 19, 'abbreviation': 'NOP', 'city': 'New...          205.0  
66  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...          181.0  
67  {'id': 28, 'abbreviation': 'TOR', 'city': 'Tor...          217.0  

[3268 rows x 8 columns]

In [113]:
players = players.rename(columns={"id":"player_id"})
players.head(5)

player_id first_name  height_feet  height_inches last_name position  \
0         14        Ike          NaN            NaN  Anigbogu        C   
1         25        Ron          NaN            NaN     Baker        G   
2         47     Jabari          NaN            NaN      Bird        G   
3         67    MarShon          NaN            NaN    Brooks        G   
4         71    Lorenzo          NaN            NaN     Brown        G   

                                                team  weight_pounds  
0  {'id': 12, 'abbreviation': 'IND', 'city': 'Ind...            NaN  
1  {'id': 20, 'abbreviation': 'NYK', 'city': 'New...            NaN  
2  {'id': 2, 'abbreviation': 'BOS', 'city': 'Bost...            NaN  
3  {'id': 15, 'abbreviation': 'MEM', 'city': 'Mem...            NaN  
4  {'id': 28, 'abbreviation': 'TOR', 'city': 'Tor...            NaN

In [117]:
players["Ht"]=players["height_feet"]*0.3048+players["height_inches"]*0.025

In [125]:
players=players.drop(columns=["height_feet","height_inches"])

In [142]:
players["team"]=players["team"].apply(lambda x: x["full_name"])

In [144]:
players["weight"]=players["weight_pounds"]*0.453

In [146]:
players=players.drop(columns="weight_pounds")

In [147]:
players.dtypes

player_id       int64
first_name     object
last_name      object
position       object
team           object
Ht            float64
weight        float64
dtype: object

In [148]:
players_season_average_2010s

games_played  player_id  season    min   fgm    fga  fg3m  fg3a   ftm  \
0             71         90    2010  23:58  3.17   7.69  1.30  3.48  0.96   
0             80        179    2010  25:24  4.23   7.54  0.01  0.05  1.75   
0             65       1593    2010  35:20  5.65  11.42  0.00  0.06  1.48   
0             72        241    2010  25:43  3.90   6.88  0.00  0.01  1.75   
0             50        263    2010   8:40  1.36   3.00  0.00  0.04  0.48   
..           ...        ...     ...    ...   ...    ...   ...   ...   ...   
0             61        212    2018  14:24  1.11   2.92  0.52  1.66  0.38   
0             63        213    2018  10:15  1.67   4.16  0.68  2.02  0.65   
0             67        214    2018  35:51  8.16  17.30  1.76  5.42  3.10   
0             82        215    2018  31:47  3.66   9.48  1.98  5.67  1.16   
0             71        216    2018  17:22  2.82   6.86  0.13  0.69  1.66   

     fta  oreb  dreb    reb   ast   stl   blk  turnover    pf    pts  fg_pct  \
0   1.42  1.04  3.28   4.32  1.04  0.76  0.18      0.96  1.62   8.59   0.412   
0   2.41  1.85  6.04   7.89  0.88  0.45  1.14      1.01  2.35  10.21   0.561   
0   3.34  3.06  8.02  11.08  2.02  0.72  2.58      1.92  3.34  12.77   0.495   
0   2.22  2.63  3.78   6.40  1.13  0.74  1.22      0.96  3.65   9.56   0.568   
0   0.86  1.14  1.46   2.60  0.14  0.20  0.50      0.62  1.18   3.20   0.453   
..   ...   ...   ...    ...   ...   ...   ...       ...   ...    ...     ...   
0   0.52  0.56  1.62   2.18  0.90  0.38  0.16      0.51  1.31   3.13   0.382   
0   0.79  0.08  0.98   1.06  1.38  0.33  0.21      0.63  1.11   4.67   0.401   
0   4.04  1.10  3.87   4.97  7.73  1.64  0.81      3.16  2.21  21.19   0.472   
0   1.29  0.56  3.38   3.94  1.78  1.48  0.44      1.27  2.01  10.45   0.386   
0   2.58  1.17  3.20   4.37  1.35  0.62  0.38      0.96  1.51   7.42   0.411   

    fg3_pct  ft_pct  
0     0.372   0.673  
0     0.250   0.725  
0     0.000   0.442  
0     0.000   0.788  
0     0.000   0.558  
..      ...     ...  
0     0.317   0.719  
0     0.339   0.820  
0     0.325   0.768  
0     0.348   0.896  
0     0.184   0.645  

[3230 rows x 22 columns]

In [149]:
players_season_average_2010s.dtypes

games_played      int64
player_id         int64
season            int64
min              object
fgm             float64
fga             float64
fg3m            float64
fg3a            float64
ftm             float64
fta             float64
oreb            float64
dreb            float64
reb             float64
ast             float64
stl             float64
blk             float64
turnover        float64
pf              float64
pts             float64
fg_pct          float64
fg3_pct         float64
ft_pct          float64
dtype: object

In [152]:
players_season_average_2010s.groupby(by="season").sum()

games_played  player_id      fgm      fga    fg3m    fg3a     ftm  \
season                                                                      
2010           18164     406427   996.65  2215.80  181.49  515.75  496.61   
2011           15259     447181  1054.17  2387.71  187.93  556.56  498.80   
2012           17430     379410   950.06  2154.32  201.13  573.47  429.28   
2013           18909     399095  1064.42  2382.52  222.34  632.76  493.36   
2014           19356     388384  1101.99  2500.60  231.58  677.24  497.30   
2015           18614     318955  1034.44  2311.67  236.98  673.78  472.49   
2016           19383     302092  1093.44  2412.60  262.89  747.97  480.52   
2017           19863     318748  1234.21  2733.85  334.44  944.89  524.31   
2018           20552    3287042  1215.02  2669.19  329.84  952.00  518.89   

           fta    oreb     dreb      reb     ast     stl     blk  turnover  \
season                                                                       
2010    658.26  300.34   840.26  1140.55  598.14  208.66  131.89    382.18   
2011    669.40  335.60   922.52  1258.05  626.21  227.27  150.02    417.55   
2012    574.03  288.70   806.27  1094.85  574.87  209.92  126.82    372.28   
2013    659.99  315.17   920.83  1235.89  631.45  217.91  138.12    415.73   
2014    675.38  328.77   950.77  1279.60  665.35  238.52  139.89    422.19   
2015    627.79  286.22   933.86  1220.13  600.61  218.10  139.89    383.45   
2016    632.60  311.92   978.57  1290.41  615.44  224.41  145.75    381.22   
2017    687.47  321.94  1079.22  1401.13  723.19  255.36  156.19    439.25   
2018    684.49  319.54  1070.71  1390.46  736.13  235.44  146.55    406.35   

            pf      pts   fg_pct  fg3_pct   ft_pct  
season                                              
2010    606.72  2671.54  143.869   76.940  230.015  
2011    603.46  2794.69  151.055   80.320  243.979  
2012    546.87  2530.64  138.024   75.823  223.962  
2013    624.43  2844.57  151.697   85.199  242.212  
2014    635.03  2932.50  156.750   91.837  253.969  
2015    588.30  2778.13  148.582   92.804  245.453  
2016    603.55  2930.06  160.156   98.577  257.161  
2017    675.01  3327.17  182.436  115.543  284.251  
2018    652.30  3278.52  173.246  114.515  278.051

In [234]:
players_season_average_1516

games_played  player_id  season  fg3m  fg3a  fg3_pct    pts
0               8         71    2015  0.13  1.00    0.125   2.50
1              69         90    2015  1.62  3.97    0.409  11.78
2              75        179    2015  0.00  0.01    0.000  13.49
3              70       1593    2015  0.01  0.01    1.000   5.36
4              79        241    2015  0.13  0.54    0.233   7.30
..            ...        ...     ...   ...   ...      ...    ...
679            80        212    2016  1.18  3.38    0.348   7.04
680            67        214    2016  1.49  4.19    0.356  15.36
681            82        215    2016  1.18  3.33    0.355   7.67
682            78        216    2016  0.19  0.86    0.224   8.65
683            79        115    2016  4.10  9.99    0.411  25.30

[684 rows x 7 columns]

## Data Storage

In [157]:
load_dotenv("./.env")

True

In [159]:
server = 'postgresql'
username = 'postgres'
password = os.getenv("password")
ip = 'localhost'
database = 'nba'

# create the engine
engine = db.create_engine(f'{server}://{username}:{password}@{ip}/{database}')

# open the connection
conn = engine.connect()

In [164]:
players_season_average_2010s.to_sql('players_season_average_2010s', con=conn, if_exists='replace', index=False)

In [165]:
players.to_sql('players', con=conn, if_exists='replace', index=False)

In [170]:
seasons.to_sql('seasons', con=conn, if_exists='replace', index=False)

<img src="NBAdbSchema.jpg" width="400" align="center">